In [ ]:
import os
import numpy as np
import h5py

from lib.parameters import Parameters
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
##  configuration options
config_file = '../parameters.xml'

cfg = Parameters(config_file)

components = ['E_r', 'E_z', 'H_phi', 'rho_beam']

start_frame = cfg.get_frame_number_by_timestamp(cfg.start_time)
end_frame = cfg.get_frame_number_by_timestamp(cfg.end_time)

main_group_name = '/pdp3/result'

enable_compression = False
compression_level = 6

In [ ]:
# define reader (plain reader used)
reader = PlainReader(cfg.data_path, cfg.system_state_path,
                     [cfg.number_r_grid , cfg.number_z_grid],
                     cfg.frames_per_file, False)
reader.verbose = False

f = h5py.File(os.path.join(cfg.data_path, "data.h5"), 'w')

In [ ]:
f.create_group(main_group_name)

In [ ]:
for compon in components:
    f.create_group(os.path.join(main_group_name, compon))

if enable_compression:
    compress='gzip'
else:
    compress=None
        
for frame in range(start_frame, end_frame):
    for compon in components:
        name = os.path.join(main_group_name, compon, str(frame))
        ds = reader.get_frame(compon, frame)
        f.create_dataset(name, data=ds, compression=compression, compression_opts=compression_level)
        print("Imported {}/{}".format(compon, frame))

f.close()